# Empathic Zoom

Este script utiliza Selenium y la librería Deep Face para detectar las caras de los participantes en una reunión de Zoom web y mostrar las emociones de la reunión.

Para ello, el script abrirá una sesión de Chrome automatizada con Selenium, accederá a la reunión de Zoom utilizando la URL proporcionada, y luego utilizará Deep Face para analizar las expresiones faciales de los participantes.

*(Podemos llegar a usar OpenCV para analizar, ademas de las emociones, los gestos corporales)*

*Por ejemplo:*
- Detección de Movimiento y Postura: Utilizar algoritmos de visión por computadora para detectar el movimiento de las personas y su postura durante la reunión. Un aumento en el movimiento o cambios en la postura pueden indicar un mayor nivel de interés o participación.
- Reconocimiento de Gestos Faciales: Analizar los gestos faciales de los participantes para detectar sonrisas, fruncir el ceño, miradas de atención, entre otros.
- Participación en la Conversación: Analizar la participación verbal de las personas en la reunión, como la cantidad de tiempo que hablan, la frecuencia con la que hacen preguntas o comentarios, etc.

#### Librerías necesarias

- pip install selenium
- pip install deepface
- pip install opencv-python

In [1]:
if False: 
    from selenium import webdriver
    from deepface import DeepFace
    import time
    import cv2

    def capture_screenshot(driver, filename='screenshot.png'):
        driver.save_screenshot(filename)

    def analyze_emotion(image_path='screenshot.png'):
        img = cv2.imread(image_path)
        return DeepFace.analyze(img, actions=['emotion'])

    def compute_engagement(emotion_results):
        engagement_scores = {
            'happy': 1,
            'surprise': 1,
            'neutral': 0.5,
            'sad': -1,
            'angry': -1,
            'fear': -1,
            'disgust': -1
        }
        
        total_score = 0
        for res in emotion_results:
            dominant_emotion = res.get('dominant_emotion')
            total_score += engagement_scores.get(dominant_emotion, 0)
        
        return total_score / len(emotion_results)

    # Initialize WebDriver
    driver = webdriver.Chrome()
    # Uso mi link personal de Zoom (corregir segun sea necesario)
    driver.get("https://app.zoom.us/wc/6566777642/join?fromPWA=1&pwd=KkJzofviLFwL1w6rUxGpAWQTDov0Zo.1&_x_zm_rtaid=UsYTRJltSqCdGRR6BBx3eA.1716752925491.af11f663de2681fb904a8c586f897e03&_x_zm_rhtaid=198")
    time.sleep(60)  # Adjust based on when the meeting starts

    while True:
        capture_screenshot(driver)
        emotion_result = analyze_emotion()
        engagement_score = compute_engagement(emotion_result)
        print(f"General Engagement Score: {engagement_score}")
        time.sleep(20)  # Capture screenshot every 20 sec

    driver.quit()

In [1]:
# Librerías necesarias

# Rutas
from pyprojroot.here import here

# Selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Detección emociones
import cv2
from deepface import DeepFace

**Importante** a partir de selenium 4 ya no es necesario descargar el driver... ver version...

Yo tengo la 4.21.0. Actualizar version...

[Link](https://stackoverflow.com/questions/76461596/unable-to-use-selenium-webdriver-getting-two-exceptions).

In [2]:
# Defino algunos parametros y rutas

# Rutas a drivers... no es utilizada por la version de Selenium... #TODO: borrarla
DRIVER_PATH = here() / "drivers"
SCREENSHOTS_PATH = here() / "screenshots" 
SCREENSHOT_FILE = SCREENSHOTS_PATH / "screenshot.png"
FACE_DETECTED_FILE = SCREENSHOTS_PATH / "faces_detected.png"

# Parámetros para pruebas en mi local...
TEST = True

In [3]:
def join_zoom_meeting():
    
    # Define meeting parameters
    if TEST:
        print("Probando desde una reunión fija...")
        zoom_params = {"browser": "edge", "meeting_id": 4702268490, "access_code": "8513"}
    else:
        zoom_params = {"browser": input("Escribir navegador: edge/chrome..."), "meeting_id": input("Escribir ID de la reunión..."), "access_code": input("Escribir Código de Acceso de la reunión...")}
        
    browser = zoom_params.get("browser")
    meeting_id = zoom_params.get("meeting_id")
    access_code = zoom_params.get("access_code")
    user_name = "Empathic Zoom Grupo 1"
    website = f"https://zoom.us/wc/join/{meeting_id}"
    join_attempts = 0
    max_join_attempts = 5
    
    # Initialize driver
    if browser=="edge":
        driver = webdriver.Edge()  # You can change this to your preferred browser
    elif browser=="chrome":
        driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(website)
    time.sleep(5)
    
    # Complete meeting parameters
    while join_attempts < max_join_attempts:
        try:
            # Complete parameters for the meeting
            # Access code
            access_code_input = driver.find_element(By.XPATH, "//*[@id='input-for-pwd']")
            access_code_input.send_keys(access_code)
            time.sleep(2)
            
            # User name
            user_name_input = driver.find_element(By.XPATH, "//*[@id='input-for-name']")
            user_name_input.send_keys(user_name)
            time.sleep(2)
            
            # Join the meeting
            driver.find_element(By.XPATH, "//*[@id='root']/div/div[1]/div/div[2]/button").click()
            time.sleep(5)
            
            # Configure audio
            while True:
                try:
                    driver.find_element(By.XPATH, "//*[@id='voip-tab']/div/button").click()
                    time.sleep(2)
                    break
                except Exception as e:
                    print(f"Error: {e}")
            
            # If everything went well, break out of the loop
            break
            
        except Exception as e:
            print(f"Error: {e}")
            join_attempts += 1

    if join_attempts == max_join_attempts:
        print("Failed after 5 attempts. Exiting...")
        time.sleep(3)  # Add a delay before quitting the driver
        driver.quit()
        
    return driver

# uso las funciones definidas por fede

def capture_screenshot(driver, filename=SCREENSHOT_FILE):
    """
    Obtener y guardar captura de pantalla con Selenium
    """
    driver.save_screenshot(filename)

def analyze_emotion(image_path=SCREENSHOT_FILE):
    """
    Analizar emociones con DeepFace
    """
    img = cv2.imread(image_path)
    return DeepFace.analyze(img, detector_backend='mtcnn', actions=['emotion']) # TODO: probar con otros modelos para detectar caras... el default no detectaba bien las caras para el archivo "screenshots\test\test_zoom_meeting.jpg"

def compute_engagement(emotion_results):
    """
    Armar índice personalizado de las emociones de la clase
    """
    engagement_scores = {
        'happy': 1,
        'surprise': 1,
        'neutral': 0.5,
        'sad': -1,
        'angry': -1,
        'fear': -1,
        'disgust': -1
    }
    
    total_score = 0
    for res in emotion_results:
        dominant_emotion = res.get('dominant_emotion')
        total_score += engagement_scores.get(dominant_emotion, 0)
    
    return total_score / len(emotion_results)

def save_detected_faces(deep_face_results, image_path = SCREENSHOT_FILE, faces_detected_path = FACE_DETECTED_FILE):
    """
    Identificar y marcar las caras detectadas 
    """
    # Cargar la imagen original
    image = cv2.imread(SCREENSHOT_FILE)
    
    # Dibujar rectángulos alrededor de las caras detectadas en una copia de la imagen original
    image_with_rectangles = image.copy()
    for face in deep_face_results:
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        cv2.rectangle(image_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    # Guardar la imagen con los rectángulos dibujados
    cv2.imwrite(FACE_DETECTED_FILE, image_with_rectangles)

## Prueba

In [4]:
# ingresar en la reunión de zoom
driver = join_zoom_meeting() # TODO: ver que hacer con los mensajes de salida...


# ir capturando las emociones cada 10 segundos
screenshot_nro = 0
screenshot_nro_max = 1
while screenshot_nro <= screenshot_nro_max:
    # tomar captura de pantalla
    capture_screenshot(driver)
    
    # obtener emociones para las caras detectadas
    if screenshot_nro == 0:
        # para el inicio del proceso
        emotion_results_0 = analyze_emotion()
        
        # Caras identificadas al inicio del proceso #TODO: ver...
        save_detected_faces(emotion_results_0, faces_detected_path = SCREENSHOTS_PATH / "faces_detected_0.png")
    
    emotion_results = analyze_emotion()
    
    # Filtrado de FOTOS (caras sin cambios)
    # Usar la clave "region" de emotion_results (ver si hacerlo con la results_0 o con la anterior)
    # Puedo comparar emotion_results y emotion_results_0 para eliminar elementos repetidos...
    
    # mostrar los resultados por cara detectada
    print(f"\nResultados de análisis de emociones para la captura de pantalla {screenshot_nro}:\n")
    print(f"\nCaras detectadas: {len(emotion_results)}\n")
    for cara_nro, emotion_result in enumerate(emotion_results):
        emotion = emotion_result['emotion']
        strongest_emotion = max(emotion, key=emotion.get)
        score = emotion[strongest_emotion]
        print(f"* CARA {cara_nro}: Emoción más fuerte: {strongest_emotion}, Score: {score}")
        # TODO: agregar emocion moda (más frecuente)
        
    
    # calcular índice personalizado de la clase
    engagement_score = compute_engagement(emotion_results)
    print(f"\nValor general del índice: {engagement_score}\n")
    # TODO: llevar a una escala 0-1
    
    # Caras identificadas al final del proceso
    if screenshot_nro == screenshot_nro_max:
        save_detected_faces(emotion_results)
    
    # esperar 10 segundos
    print("\n##################\n")
    time.sleep(10)  
    
    # pasar a la siguiente iteración
    screenshot_nro += 1
    
# cerrar driver
driver.quit() 

Probando desde una reunión fija...
Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='voip-tab']/div/button"}
  (Session info: MicrosoftEdge=125.0.2535.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71CBA5DC2+61250]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF71CB306C9+206985]
	(No symbol) [0x00007FF71C93E3C7]
	(No symbol) [0x00007FF71C9852A3]
	(No symbol) [0x00007FF71C985366]
	(No symbol) [0x00007FF71C9C02C7]
	(No symbol) [0x00007FF71C9A493F]
	(No symbol) [0x00007FF71C97AAA7]
	(No symbol) [0x00007FF71C9BDEB7]
	(No symbol) [0x00007FF71C9A4563]
	(No symbol) [0x00007FF71C979FCE]
	(No symbol) [0x00007FF71C97918C]
	(No symbol) [0x00007FF71C979B81]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF71CD5DF54+1072532]
	(No symbol) [0x00007FF71C9F2D9C]
	Micro

# Posibles pendiente:
* Estaría bueno detectar fotos... son aquellas caras que no se modifican entre distintos screenshots.
* Estaría bueno registras las emociones detectadas durante toda la clase en un archivo plano para un análisis posterior del mismo.
* Podemos probar con los distintos métodos de detección de caras, esta seteado con "mtcnn", pero hay varias opciones:

    backends = [
    'opencv', 
    'ssd', 
    'dlib', 
    'mtcnn', 
    'fastmtcnn',
    'retinaface', 
    'mediapipe',
    'yolov8',
    'yunet',
    'centerface',
    ]

* Podemos guardar las caras detectadas para el cálculo del índice.

In [6]:
# Prueba detección de caras

if True:
    import cv2
    from deepface import DeepFace

    # Cargar la imagen original
    #image_path = 'screenshots/test/test_zoom_meeting.jpg'
    image_path = 'screenshots/test/prueba_clase.jpeg'
    image = cv2.imread(image_path)

    # Analizar las emociones en la imagen usando MTCNN para la detección facial
    resultados = DeepFace.analyze(image_path, detector_backend='mtcnn', actions=['emotion']) #yolov8 mtcnn
    #resultados = DeepFace.analyze(image_path, actions=['emotion'])

    # Dibujar rectángulos alrededor de las caras detectadas en una copia de la imagen original
    image_with_rectangles = image.copy()
    for face in resultados:
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        cv2.rectangle(image_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # TODO: agregar etiqueta persona

    # Guardar la imagen con los rectángulos dibujados
    cv2.imwrite('faces_detected.jpg', image_with_rectangles)
    
    print(f"\nCaras detectadas: {len(resultados)}\n")
    for cara_nro, emotion_result in enumerate(resultados):
        emotion = emotion_result['emotion']
        strongest_emotion = max(emotion, key=emotion.get)
        score = emotion[strongest_emotion]
        print(f"* CARA {cara_nro}: Emoción más fuerte: {strongest_emotion}, Score: {score}")
        
    engagement_score = compute_engagement(resultados)
    print(f"\nValor general del índice: {engagement_score}\n")

    # Mostrar la imagen con los rectángulos dibujados
    #cv2.imshow('Faces Detected', image_with_rectangles)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

3/3 [==============================] - 0s 27ms/step

Caras detectadas: 18

* CARA 0: Emoción más fuerte: neutral, Score: 44.70013976097107
* CARA 1: Emoción más fuerte: happy, Score: 98.76717329025269
* CARA 2: Emoción más fuerte: neutral, Score: 80.26928901672363
* CARA 3: Emoción más fuerte: neutral, Score: 74.42311644554138
* CARA 4: Emoción más fuerte: sad, Score: 52.59696841239929
* CARA 5: Emoción más fuerte: angry, Score: 53.35234999656677
* CARA 6: Emoción más fuerte: sad, Score: 44.812446523454575
* CARA 7: Emoción más fuerte: sad, Score: 98.87004494667053
* CARA 8: Emoción más fuerte: angry, Score: 90.58168530464172
* CARA 9: Emoción más fuerte: sad, Score: 49.34665262699127
* CARA 10: Emoción más fuerte: neutral, Score: 94.18079818372716
* CARA 11: Emoción más fuerte: neutral, Score: 91.61514639854431
* CARA 12: Emoción más fuerte: neutral, Score: 43.15902251953021
* CARA 13: Emoción más fuerte: fear, Score: 42.85517930984497
* CARA 14: Emoción más fuerte: sad, Score: 46.787

In [7]:
resultados

[{'emotion': {'angry': 2.8288209810853004,
   'disgust': 0.03899333823937923,
   'fear': 29.136261343955994,
   'happy': 15.283671021461487,
   'sad': 7.353372126817703,
   'surprise': 0.6587453652173281,
   'neutral': 44.70013976097107},
  'dominant_emotion': 'neutral',
  'region': {'x': 761,
   'y': 355,
   'w': 50,
   'h': 62,
   'left_eye': (796, 377),
   'right_eye': (771, 377)},
  'face_confidence': 1.0},
 {'emotion': {'angry': 1.1088157072663307,
   'disgust': 1.9477361945519078e-07,
   'fear': 0.11030178284272552,
   'happy': 98.76717329025269,
   'sad': 3.288897971742699e-05,
   'surprise': 0.013183134433347732,
   'neutral': 0.0004886753686150769},
  'dominant_emotion': 'happy',
  'region': {'x': 1431,
   'y': 509,
   'w': 45,
   'h': 58,
   'left_eye': (1464, 531),
   'right_eye': (1443, 531)},
  'face_confidence': 1.0},
 {'emotion': {'angry': 15.486347675323486,
   'disgust': 0.00022706024083163356,
   'fear': 0.09140910115092993,
   'happy': 0.15492161037400365,
   'sad': 